In [ ]:
import transformers
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
# from tensorflow.keras import backend as K
import os
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from transformers import  DistilBertConfig, TFDistilBertModel
from transformers import RobertaConfig, TFRobertaModel
import tensorflow as tf 
from tensorflow.compat.v1 import InteractiveSession
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.compat.v1.Session(config=config)
from textattack.augmentation import WordNetAugmenter
from textattack.augmentation import EmbeddingAugmenter
from sklearn.metrics import classification_report
from matplotlib.ticker import StrMethodFormatter
from tensorflow.keras.callbacks import History
from keras_one_cycle_clr import keras_one_cycle_clr as ktool
import CLR
from tensorflow.keras.callbacks import History
ROOT_DIR = os.path.dirname(os.path.abspath('.')) 
data_dir = ROOT_DIR + '\\{}\\{}\\{}\\'.format('data', 'Embeddings_output', 'Irony_Data')

### Import training data

In [ ]:
data = pd.read_csv('../data/final_training_irony.csv')

In [ ]:
data['text'] = data["text"].str.replace(r'@([^\s:]+)', '')
data['text'] = data['text'].str.replace(r'#([^\s:]+)', '')

In [ ]:
import re
#pulizia url classico
def remove_link(x):
    text = re.sub(r'^https?:\/\/.[\r\n]', '', str(x), flags=re.MULTILINE)
    return text
#rimozione definitiva di tutti i link 

def remove_link2(x):
    text = re.sub(r'http\S+', '', x)
    return text

def remove_rest(x):
    text = re.sub(r':\S+', "", x)
    return text

In [ ]:
data['text'] = data['text'].apply(remove_link)
data['text'] = data['text'].apply(remove_link2)
data['text'] = data['text'].apply(remove_rest)

In [ ]:
data = data[(data.text != ' ') & (data.text != '')]

### Importing transformers

In [ ]:
from transformers import DistilBertTokenizer, DistilBertConfig
from transformers import RobertaTokenizer, RobertaConfig

In [ ]:
roberta = 'distilbert-base-uncased'
# Defining RoBERTa tokinizer
tokenizer = DistilBertTokenizer.from_pretrained(roberta, do_lower_case=True, add_special_tokens=True,
                                                max_length=20, pad_to_max_length=True)

In [ ]:
roberta = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(roberta, do_lower_case=True, add_special_tokens=True,
                                                max_length=20, pad_to_max_length=True)

In [ ]:
def tokenize(sentences, tokenizer):
    input_ids, input_masks, input_segments = [],[],[]
    for sentence in tqdm(sentences):
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=20, pad_to_max_length=True, 
                                             return_attention_mask=True, return_token_type_ids=True,truncation=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')

In [ ]:
input_id, input_mask, input_seg = tokenize(train_txt.text, tokenizer)

In [ ]:
distil_bert = 'distilbert-base-uncased'
roberta = 'roberta-base'
config = RobertaConfig(dropout=0.4, attention_dropout=0.4)
config.output_hidden_states = True
#transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)
config = RobertaConfig.from_pretrained(roberta, output_hidden_states=True)
config.output_hidden_states = True
transformer_model = TFRobertaModel.from_pretrained(roberta, config = config)

### Embedding extraction from transformer

In [ ]:
# input_ids_in = tf.keras.layers.Input(shape=(512,), name='input_token', dtype='int32')
# input_masks_in = tf.keras.layers.Input(shape=(512,), name='masked_token', dtype='int32') 
# #input_seg_in = tf.keras.layers.Input(shape = (1130,), name = 'segment_id', dtype = 'int32')
# embedding_layer = transformer_model([input_ids_in, input_masks_in])[0] 


In [ ]:
input_ids_in = tf.keras.layers.Input(shape=(20,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(20,), name='masked_token', dtype='int32') 

embedding_layer = transformer_model([input_ids_in, input_masks_in])[0] 

model2 = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = embedding_layer)

#### Generator for extract embeddings

In [ ]:
def transformer_batch_generator(input_id, input_mask, y, batch_size=1):
    while True:
        n_batches_per_epoch = input_id.shape[0]//batch_size
        for i in range(n_batches_per_epoch):
            index_batch = range(input_id.shape[0])[batch_size*i:batch_size*(i+1)]       
            x_batch_id = input_id[index_batch,:]
            x_batch_mask = input_mask[index_batch,:]
            y_batch = y[index_batch,:]
            yield [x_batch_id, x_batch_mask], np.array(y_batch)

In [ ]:
y_train = tf.keras.utils.to_categorical(
    np.array(train_txt.label), num_classes=2, dtype='float32')

### Prediction one by one document and storage each array directly to a folder

In [ ]:
#id sequenziali 
for i in tqdm(range(input_id.shape[0])):
    output = model2.predict([input_id[i,:].reshape(1,-1), input_mask[i, :].reshape(1,-1)], verbose = 0)
    
    np.save('../data/Embeddings_output/Irony_Data/doc_{}.npy'.format(i), output)
    np.save('../data/Embeddings_output/Irony_Data/target_{}.npy'.format(i), y_train[i,:])

#### Generator flow from directory

In [ ]:
def train_batch_generator(batch_size = 64):
    
#     while True:    
    folder = os.listdir(data_dir)
    n_batches_per_epoch = (y_train.shape[0] - 4000)//batch_size
    for i in range(n_batches_per_epoch):

        index_batch = range(4000, len(y_train))[batch_size*i:batch_size*(i+1)]
        batch = [np.load('{}\\{}'.format(data_dir, 'doc_{}.npy'.format(i))) for i in index_batch]

        x_batch = np.squeeze(np.asarray(batch))
        y_batch = [np.load('{}\\{}'.format(data_dir, 'target_{}.npy'.format(i))) for i in index_batch]

        yield x_batch, np.array(y_batch)

In [ ]:
def valid_batch_generator(batch_size = 64):
    
#     while True:
    folder = os.listdir(data_dir[:73] + '\\Irony_validation\\')

    n_batches_per_epoch = 4000//batch_size

    for i in range(n_batches_per_epoch):

        index_batch = range(4000)[batch_size*i:batch_size*(i+1)]
        batch = [np.load('{}\\{}'.format(data_dir[:73] + '\\Irony_validation', 'doc_{}.npy'.format(i))) for i in index_batch]

        x_batch = np.squeeze(np.asarray(batch))
        y_batch = [np.load('{}\\{}'.format(data_dir[:73] + '\\Irony_validation', 'target_{}.npy'.format(i))) for i in index_batch]

        yield x_batch, np.array(y_batch)

### Final model

In [ ]:
mcp_save = tf.keras.callbacks.ModelCheckpoint('.mdl_last_transformer.hdf5', save_best_only=True, monitor='train_loss', mode='min')

In [ ]:
input_layer = tf.keras.layers.Input(shape=(20, 768), name='input_emb')
#norm = tf.keras.layers.BatchNormalization()(input_layer)
X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(input_layer)
X = tf.keras.layers.Concatenate(axis=-1)([X, input_layer])
X = tf.keras.layers.MaxPooling1D(20)(X)
X = tf.keras.layers.SpatialDropout1D(0.4)(X)
X = tf.keras.layers.Flatten()(X)

#X = tf.keras.layers.Dense(512, activation="relu")(X)
#X = tf.keras.layers.Dropout(0.4)(X)
X = tf.keras.layers.Dense(128, activation="relu")(X)
X = tf.keras.layers.Dropout(0.25)(X)
X = tf.keras.layers.Dense(2, activation='softmax')(X)

model = tf.keras.Model(inputs=input_layer, outputs = X)

In [ ]:
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate= 2e-5, epsilon= 1e-3)
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer= opt)

In [ ]:
clr = CLR.CyclicLR(base_lr=0.00001, max_lr=0.01,
                        step_size=(len(y_train) - 15000)//50)

In [ ]:
history = History()

In [ ]:
model.fit(train_batch_generator(50), epochs=25, steps_per_epoch=(len(y_train) - 4000)//50, verbose=1, callbacks=[history], validation_data= valid_batch_generator(50),validation_steps=4000//50)

In [ ]:
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.figure(figsize = (5,5))
plt.grid(linestyle = 'dashed')
plt.plot(history.history['acc'], c = 'green')
plt.plot(history.history['val_acc'])
plt.title('Accuracy Metric with respect to RCNN-Roberta')
plt.legend(['Train Set', 'Validation Set'])
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.tight_layout()
plt.savefig(r'D:\Data_Science_all\MSC_2_anno\Tesi_Irony_Sarcasm\thesis_latex\img\accuracy_trasformer_irony.png', dpi=500)

In [ ]:
# "Loss"
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.figure(figsize = (5,5))
plt.grid(linestyle = 'dashed')
plt.plot(history.history['loss'], c = 'green')
plt.plot(history.history['val_loss'])
plt.title('Cross Entropy Loss with respect to RCNN-Roberta')
plt.legend(['Train Set', 'Validation Set'])
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
plt.tight_layout()
plt.savefig(r'D:\Data_Science_all\MSC_2_anno\Tesi_Irony_Sarcasm\thesis_latex\img\loss_trasformer_irony.png', dpi=500)

In [ ]:
model.save_weights("model_irony.h5")

In [ ]:
model.load_weights('model_irony.h5')

In [ ]:
train_prediction = model.predict(train_batch_generator(4), verbose = 1)

In [ ]:
valid_prediction = model.predict(valid_batch_generator(5), verbose = 1)

#### Test set

In [ ]:

sem = pd.read_csv('../data/SemEval2018-Task3/datasets/goldtest_TaskA/SemEval2018-T3_gold_test_taskA_emoji.txt', sep='\t')
sem.rename({'Tweet text': 'text', 'Label' : 'label'}, axis = 1, inplace=True)

In [ ]:
def text_prepro(data):
    
    data['text'] = data["text"].str.replace(r'@([^\s:]+)', '')
    data['text'] = data['text'].str.replace(r'#([^\s:]+)', '')
    data['text'] = data['text'].apply(remove_link)
    data['text'] = data['text'].apply(remove_link2)
    data['text'] = data['text'].apply(remove_rest)
#     data['text'] = data['text'].apply(lambda x: x.lower())
    
    input_id, input_mask, input_seg = tokenize(data.text, tokenizer)
    
    return input_id, input_mask, input_seg

In [ ]:
input_id_sem, input_mask_sem, input_seg_sem = text_prepro(sem)

In [ ]:
for i in tqdm(range(input_id_sem.shape[0])):
    output = model2.predict([input_id_sem[i,:].reshape(1,-1), input_mask_sem[i, :].reshape(1,-1)], verbose = 0)
    
    np.save('../data/Embeddings_output/Irony_test/doc_{}.npy'.format(i), output)
    np.save('../data/Embeddings_output/Irony_test/target_{}.npy'.format(i), sem.label[i])

In [ ]:
data_dir = ROOT_DIR + '\\{}\\{}\\{}\\'.format('data', 'Embeddings_output', 'Irony_test')

In [ ]:
def test_batch_generator(batch_size = 784, n_batches = 64):
        
        data_dir2 = data_dir

        n_batches_per_epoch = n_batches//batch_size

        for i in range(n_batches_per_epoch):

            index_batch = range(n_batches)[batch_size*i:batch_size*(i+1)]
            batch = [np.load('{}\\{}'.format(data_dir2, 'doc_{}.npy'.format(i))) for i in index_batch]

            x_batch = np.squeeze(np.asarray(batch))
            y_batch = [np.load('{}\\{}'.format(data_dir2, 'target_{}.npy'.format(i))) for i in index_batch]

            yield x_batch, np.array(y_batch)

In [ ]:
test_prediction = model.predict(test_batch_generator(2, len(sem.label)), verbose = 1)

In [ ]:
print(classification_report(test_prediction.argmax(axis = 1), sem.label))

In [ ]:
def normalize_lab(x):
    new_lab_list = []
    for i in x:
        if i == 0:
            new_lab = '1:1'
        else:
            new_lab =  '2:0'
            
        new_lab_list.append(new_lab)
        
    return new_lab_list

def mark_error(actual, predicted):
    mark_list = []
    for i,j in zip(actual, predicted):
        if i != j:
            mark = '+'
        else:
            mark = np.nan
            
        mark_list.append(mark)
        
    return mark_list

def get_proba_distrib(clf_proba):
    proba_ast = []
    for i,j in zip(clf_proba[:,0], clf_proba[:,1]):
        if i > j:
            proba = ['*{}'.format(str(i.round(5))), str(j.round(5))]
        else:
            proba = [str(i.round(5)), '*{}'.format(str(j.round(5)))]
            
        proba_ast.append(proba)
        
    return np.array(proba_ast)

In [ ]:
def get_outpupt_bma(pred,proba, ground_truth):
    
    
    actual = normalize_lab(ground_truth)
    predicted = normalize_lab(pred)
    
    error = mark_error(actual, predicted)
    conta = 0
    lista_ins = []
    for i in range(len(ground_truth)):
        conta += 1
        if conta == int(len(ground_truth)/10) + 2:
            conta = 1
     
        lista_ins.append(conta)  
        
    instanc = lista_ins
    
    predict_proba = proba
    
    distribution = get_proba_distrib(predict_proba)
    
    final_df = pd.DataFrame(instanc, columns=['inst#'])
    
    final_df['actual'] = actual
    
    final_df['predicted'] = predicted
    
    final_df['error'] = error

    final_df['distribution'] = distribution[:, 0]
    
    final_df[''] = distribution[:,1]
    
    return final_df

In [ ]:
df = get_outpupt_bma(test_prediction.argmax(axis = 1), test_prediction,  sem.label)

In [ ]:
df.to_csv(r'D:\Data_Science_all\MSC_2_anno\Tesi_Irony_Sarcasm\Code\BMA\results_semeval\input\prediction_file\transformer_model_based.csv', index = False)